
# Structural frequency primer

## Objectives
- Characterise how the structural frequency νf amplifies ΔNFR to evolve the Primary Information Structure (EPI).
- Show how deterministic ΔNFR hooks orchestrate νf drift while respecting the nodal equation ∂EPI/∂t = νf · ΔNFR.
- Provide a minimal trace that Phase-2 automation can reuse as a regression guard for frequency regulation.

## Phase-2 dependencies
- [Phase-2 integration notes](../fase2_integration.md) — the primer mirrors the scripted hooks expected by the automated ΔNFR orchestration pipeline.
- :mod:`tnfr.dynamics.adaptation` — νf adaptation rules remain the same once the Phase-2 controller swaps in its adaptive gains.
- :mod:`tnfr.structural` — operator sequencing is shared across the canonical demos and the Phase-2 integration plan.

## Theoretical exposition
The nodal equation couples ΔNFR with νf so that every glyph application scales reorganisations by the current structural frequency. Holding ΔNFR constant while perturbing νf therefore generates measurable changes in ∂EPI/∂t. The primer isolates this dependency: a scripted hook records νf before each glyph, applies ΔNFR, and updates EPI by νf · ΔNFR so the trajectory remains canonical. Tracking the before/after νf pairs surfaces how even a small deterministic drift reshapes the coherence throughput.

## Deterministic smoke check
The code cell below instantiates a node, installs a reproducible ΔNFR hook, and fires the canonical emission→reception→coherence→resonance→transition sequence. The hook captures νf prior to each glyph so we can assert that the EPI increment equals νf · ΔNFR at every step. The resulting timeline doubles as a CI smoke test: any future change that perturbs the νf multiplier or the hook contract will alter the recorded derivatives.


In [ ]:

from tnfr.constants import DNFR_PRIMARY, EPI_PRIMARY, VF_PRIMARY
from tnfr.dynamics import set_delta_nfr_hook
from tnfr.structural import Coherence, Emission, Reception, Resonance, Transition, create_nfr, run_sequence

G, node = create_nfr("frequency-smoke", epi=0.42, vf=1.8, theta=0.0)
trajectory: list[dict[str, float]] = []
increments = iter([0.035, 0.028, 0.024, 0.02, 0.018])

def scripted_delta(graph):
    dnfr = next(increments, 0.015)
    nd = graph.nodes[node]
    vf_before = float(nd[VF_PRIMARY])
    epi_before = float(nd[EPI_PRIMARY])
    nd[DNFR_PRIMARY] = dnfr
    nd[EPI_PRIMARY] = epi_before + vf_before * dnfr
    nd[VF_PRIMARY] = vf_before + 0.05 * dnfr
    trajectory.append(
        {
            "νf_before": round(vf_before, 6),
            "ΔNFR": round(dnfr, 6),
            "EPI": round(nd[EPI_PRIMARY], 6),
            "∂EPI/∂t": round(vf_before * dnfr, 6),
            "νf_after": round(nd[VF_PRIMARY], 6),
        }
    )

set_delta_nfr_hook(G, scripted_delta, note="structural frequency primer smoke")
run_sequence(G, node, [Emission(), Reception(), Coherence(), Resonance(), Transition()])

trajectory
